In [1]:
import mysql.connector

mydb = mysql.connector.connect(
  host="cowstudio.wayne-lee.cn",
  user="cowstudio",
  password="cowstudio_2119",
  database="cowstudio"
)

In [2]:
from pyspark import SparkConf, SparkContext
import os
os.environ['PYSPARK_PYTHON'] = "/opt/conda3/envs/lab2/bin/python"
conf = SparkConf().setAppName("jieba_test2").setMaster("spark://node01:10077")\
                  .set("spark.driver.memory", "2g")\
                  .set("spark.executor.memory", "4g")\
                  .set("spark.cores.max", "3")\
                  .set("spark.sql.shuffle.partitions", "12")\
                  .set("spark.sql.autoBroadcastJoinThreshold", "-1")
sc = SparkContext(conf=conf)

23/04/18 09:47:01 WARN util.NativeCodeLoader: Unable to load native-hadoop library for your platform... using builtin-java classes where applicable
Setting default log level to "WARN".
To adjust logging level use sc.setLogLevel(newLevel). For SparkR, use setLogLevel(newLevel).
23/04/18 09:47:03 WARN spark.SparkContext: Please ensure that the number of slots available on your executors is limited by the number of cores to task cpus and not another custom resource. If cores is not the limiting resource then dynamic allocation will not work properly!


In [9]:
import jieba
import jieba.analyse

stop_words_rdd = sc.textFile("hdfs:///user/spark_temp/stopwords.dat")
stop_words_set = set(stop_words_rdd.collect())

def cut_name_and_desc(item):
    id, name, desc = item
    name_cut = set(jieba.cut(name))
    name_cut_pure = set(jieba.cut(name)) - stop_words_set
    desc_cut = set(jieba.cut(desc))
    desc_cut_pure = set(jieba.cut(desc)) - stop_words_set
    return (id,name_cut,name_cut_pure,desc_cut,desc_cut_pure)

def to_count(item):
    id,name_cut,name_cut_pure,desc_cut,desc_cut_pure = item
    for i in name_cut_pure:
        yield (i,1)
    for i in desc_cut_pure:
        yield (i,1)

In [4]:
cur = mydb.cursor()

In [17]:
cur.reset()
cur.execute("SELECT id,name,description from items order by RAND() ")
result = cur.fetchall()
# print(result)

In [18]:
all_items_rdd = sc.parallelize(result)
# print(all_items_rdd.collect())

In [19]:

items_cut_rdd = all_items_rdd.map(cut_name_and_desc)
# print(rdd2.collect())

In [20]:
words_to_count = items_cut_rdd.flatMap(to_count)
# print(words_to_count.collect())

In [21]:
words_count = words_to_count.reduceByKey(lambda a,b:a+b)
print(words_count.collect())

[('口感', 91), ('鲜美', 87), ('精选', 14), ('疫苗', 5), ('身体健康', 3), ('确保', 5), ('管理', 7), ('用户', 6), ('黑牛', 34), ('温和', 17), ('设计', 3), ('量身', 1), ('租车', 2), ('自驾游', 2), ('车载', 1), ('做', 11), ('略带', 1), ('加工', 17), ('智能化', 2), ('实时', 3), ('技术培训', 2), ('养殖', 32), ('公司', 2), ('蛙', 1), ('实惠', 7), ('西门', 13), ('火锅', 20), ('黑', 25), ('原料', 10), ('饲', 8), ('回味', 1), ('肉为', 3), ('三年', 3), ('牛用', 11), ('症状', 2), ('疾病', 10), ('太行山', 2), ('艾丽丝', 2), ('品质', 11), ('公斤', 5), ('美味', 15), ('重约', 2), ('菜品', 2), ('制作', 19), ('烤牛肉', 1), ('河豚', 2), ('土', 2), ('辽宁省', 36), ('特有', 5), ('味道鲜美', 7), ('小黑', 3), ('位于', 2), ('400g', 1), ('腰花', 1), ('回味无穷', 10), ('花', 2), ('工艺', 3), ('油', 1), ('古老', 2), ('化', 1), ('质量', 4), ('乳制品', 6), ('肉用', 5), ('柔软', 3), ('手感', 1), ('手套', 1), ('质地', 1), ('皮带', 1), ('花蹄', 1), ('花蹄肉', 1), ('记录', 2), ('产品质量', 1), ('平原', 1), ('典型', 2), ('母牛', 27), ('双用', 1), ('草原', 32), ('塔尔牛', 8), ('分布', 6), ('原味', 4), ('牛蹄', 5), ('鲜香', 6), ('全国', 1), ('送达', 3), ('快速', 8), ('乳', 1), ('冷', 1), ('毛色', 3),